In [3]:
import pandas as pd
import numpy as np
import os
import glob

datadir = "C:/Users/jdhuang/OneDrive - Colorado School of Mines/Research/MIDDMI/TCO/Data"
os.chdir(datadir)

ignore_lib = ['temp'] #libraries to ignore

#load variable definitions
vardictdf = pd.read_excel('VariableDict.xlsx')
#put regular variables into dict
regvar = vardictdf[vardictdf.Keyword.str[0] != '!']
varnames = list(regvar.VarName)
headers = list(regvar.Keyword + ': ' + regvar.DisplayName)
vardict = dict(zip(varnames,headers))    
#composition variables
compvar = vardictdf[vardictdf.Keyword == '!composition']

In [125]:
os.chdir(datadir)

libprefix = 'PDAC_COM3_'

for lib in next(os.walk(datadir))[1]:
    if lib not in ignore_lib:
        os.chdir(lib)
        all_var = pd.DataFrame([])
        #ignore_fnames = [lib + '_' + var + '.txt' for var in ignore_var] #files to ignore
        pts = pd.read_csv(lib + '_Points.txt',sep='\t',usecols = [0]) #read the Points file to determine file length
        #dataframes to store composition data
        chm = pd.DataFrame()#np.zeros((len(pts),1),dtype='str'),columns=['formula'])
        sitesum = pd.DataFrame(np.zeros((len(pts),2)), columns = ['A','B'])
        
        for fname in glob.glob(lib + '*.txt'):
            vname = fname[len(lib)+1:fname.find('.txt')]
            #regular variables
            if vname in list(vardict.keys()):
                if len(all_var) == 0:
                    cols = None
                else:
                    cols = [3]
                df = pd.read_csv(fname,sep='\t',usecols = cols)
                all_var = pd.concat([all_var,df], axis=1)
            #composition variables
            elif vname in list(compvar.VarName):
                dfc = pd.read_csv(fname,sep='\t',usecols = [3])
                elmnt = vname[:vname.find('_at')]
                chm[elmnt] = dfc[vname] #chm.formula + elmnt + dfc[vname].astype(str)
                #chm[elmnt + 'name'] = elmnt
                sitesum.A = sitesum.A + dfc[vname]
                
        #determine composition
        sitesum.B = 1 - sitesum.A
        sitemax = sitesum.max(axis=1)
        sitenorm = sitesum.divide(sitemax,axis=0) #normalize for the higher occupancy site
        chmnorm = chm.divide(sitemax,axis=0)
        chmnorm.insert(0,'Ba',list(sitenorm.B))
        chmnorm = chmnorm.round(5)
        for elmnt in list(chmnorm.columns):
            chmnorm[elmnt] = elmnt + chmnorm[elmnt].map(str) 
        chmnorm['formula'] = chmnorm.apply(lambda x: ''.join(x),axis=1)
        chmnorm['formula'] = chmnorm['formula'] + 'O3'
        
        sample = lib[len(libprefix):]
        
        all_var = all_var.rename(index=str, columns = vardict) #rename headers to ingestable format using vardict
        all_var.insert(0,'IDENTIFIER: Sample number',sample) #add sample column        
        all_var.insert(0,'FORMULA',list(chmnorm['formula'])) #add formula column
        all_var.to_csv(lib + '_AllVar.csv',index=False)
        os.chdir('..')

In [127]:
all_var

,FORMULA,IDENTIFIER: Sample number,IDENTIFIER: Point,IDENTIFIER: Row,IDENTIFIER: Column,PROPERTY: Cubic lattice constant (A),PROPERTY: Film thickness (um),PROPERTY: FWHM of 110 peak (º),PROPERTY: FWHM of 111 peak (º),PROPERTY: FWHM of 112 peak (º),...,PROPERTY: FWHM ITO Peak 2,PROPERTY: Mak Intensity 110,PROPERTY: Max Intensity 111,PROPERTY: Max intensity of 200 peak (counts),PROPERTY: Position of 110 peak (º),PROPERTY: Position of 111 peak (º),PROPERTY: Position of 200 peak (º),IDENTIFIER: X position (mm),IDENTIFIER: Y position (mm),PROPERTY: ZrY at %
0,Ba0.93613Co0.87905Fe0.0Y0.0Zr0.12095O3,01140,1,1,1,4.099105,0.31069,0.541012,0.442743,0.571084,...,0,4174.0913,1926.55600,1712.95360,30.811657,37.889252,44.088844,5.400000,44.142010,0
1,Ba0.92792Co0.88134Fe0.0Y0.0Zr0.11866O3,01140,2,1,2,4.090169,0.31546,0.506540,0.476404,0.784516,...,0,5210.8438,2039.96570,965.28607,30.880646,38.015530,44.135906,9.398000,44.142010,0
2,Ba0.92343Co0.87899Fe0.0Y0.0Zr0.12101O3,01140,3,1,3,4.084676,0.31389,0.505416,0.479806,0.753538,...,0,5545.4653,1952.60800,752.65771,30.923212,38.056389,44.215134,13.395990,44.142010,0
3,Ba0.91563Co0.87689Fe0.0Y0.0Zr0.12311O3,01140,4,1,4,4.080762,0.30516,0.525699,0.506159,0.777701,...,0,5060.5444,1363.86600,995.82074,30.953617,38.102352,44.271595,17.393999,44.142010,0
4,Ba0.92706Co0.88523Fe0.0Y0.0Zr0.11477O3,01140,5,1,5,4.082595,0.29185,0.533234,0.508591,0.824508,...,0,4753.4243,1454.38370,868.68018,30.939373,38.085484,44.254524,21.391991,44.142010,0
5,Ba0.93221Co0.88595Fe0.0Y0.0Zr0.11405O3,01140,6,1,6,4.084432,0.27475,0.545245,0.513721,0.778735,...,0,4216.3301,1433.01450,756.80078,30.925112,38.052181,44.250530,25.389999,44.142010,0
6,Ba0.92189Co0.88362Fe0.0Y0.0Zr0.11638O3,01140,7,1,7,4.088128,0.25299,0.550316,0.539418,0.774449,...,0,3761.6697,1242.60990,684.26373,30.896450,38.027500,44.177029,29.388020,44.142010,0
7,Ba0.9287Co0.88964Fe0.0Y0.0Zr0.11036O3,01140,8,1,8,4.091872,0.22925,0.562578,0.564094,0.720904,...,0,3141.9326,1045.66670,756.68896,30.867479,37.981762,44.113560,33.386009,44.142010,0
8,Ba0.93227Co0.89015Fe0.0Y0.0Zr0.10985O3,01140,9,1,9,4.098338,0.20656,0.612094,0.643689,0.663699,...,0,2303.7874,671.06146,1066.52150,30.817568,37.908726,44.039253,37.384010,44.142010,0
9,Ba0.94609Co0.88703Fe0.0Y0.0Zr0.11297O3,01140,10,1,10,4.095802,0.18348,0.633340,0.531254,NaN,...,0,2021.9868,1579.29390,NaN,30.837118,37.907166,NaN,41.381989,44.142010,0


In [93]:
list(formula)

['Co0.87905135501079Fe0.0Y0.0Zr0.12094864498921004',
 'Co0.8813409686118996Fe0.0Y0.0Zr0.11865903138810045',
 'Co0.8789928233723414Fe0.0Y0.0Zr0.12100717662765859',
 'Co0.8768900018722613Fe0.0Y0.0Zr0.12310999812773878',
 'Co0.8852347899055771Fe0.0Y0.0Zr0.11476521009442289',
 'Co0.885946412774028Fe0.0Y0.0Zr0.11405358722597211',
 'Co0.8836230971587281Fe0.0Y0.0Zr0.11637690284127192',
 'Co0.8896415282914353Fe0.0Y0.0Zr0.11035847170856475',
 'Co0.8901477255688204Fe0.0Y0.0Zr0.10985227443117966',
 'Co0.8870294440647075Fe0.0Y0.0Zr0.11297055593529247',
 'Co0.8861971000452336Fe0.0Y0.0Zr0.11380289995476645',
 'Co0.8749449286210171Fe0.0Y0.0Zr0.1250550713789829',
 'Co0.8783146398076195Fe0.0Y0.0Zr0.1216853601923806',
 'Co0.8795967295715208Fe0.0Y0.0Zr0.12040327042847927',
 'Co0.8789872268503491Fe0.0Y0.0Zr0.12101277314965089',
 'Co0.8764809723890068Fe0.0Y0.0Zr0.12351902761099326',
 'Co0.8778833333462569Fe0.0Y0.0Zr0.1221166666537431',
 'Co0.8816701186703632Fe0.0Y0.0Zr0.11832988132963693',
 'Co0.8805777368